In [1]:
import os
api_key = ''

In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
#tool creation

@tool
def multiply(a:int,b:int)->int:
    '''multiply two number'''
    return a*b

In [4]:
print(multiply.invoke({'a':3,'b':8}))

24


In [5]:
llm = HuggingFaceEndpoint(
    repo_id = 'Qwen/Qwen3-235B-A22B',
    task = 'text-generation',
    huggingfacehub_api_token=api_key
)

c_llm = ChatHuggingFace(llm=llm)

In [6]:
c_llm_with_tools = c_llm.bind_tools([multiply])

In [7]:
c_llm_with_tools

RunnableBinding(bound=ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id='Qwen/Qwen3-235B-A22B', huggingfacehub_api_token='', stop_sequences=[], server_kwargs={}, model_kwargs={}, model='Qwen/Qwen3-235B-A22B', client=<InferenceClient(model='Qwen/Qwen3-235B-A22B', timeout=120)>, async_client=<InferenceClient(model='Qwen/Qwen3-235B-A22B', timeout=120)>, task='text-generation'), model_id='Qwen/Qwen3-235B-A22B', model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply two number', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [17]:
c_llm_with_tools.invoke('how are you, and what you can do')

AIMessage(content='<think>\nOkay, the user asked, "how are you, and what you can do." Let me break this down.\n\nFirst, they\'re asking about my well-being, which is a common greeting. Then, they want to know what I can do. Since I\'m an AI, I should respond politely and explain my capabilities.\n\nLooking at the tools provided, there\'s a multiply function. But the user\'s question isn\'t about multiplying numbers. They’re asking about my functionality in general. So, I don\'t need to use the tool here. My response should be a friendly message detailing what I can assist with, like answering questions, providing information, performing calculations, etc.\n\nI should make sure to acknowledge their greeting and list some examples of tasks I can handle. No function call required for this query.\n</think>\n\nI\'m functioning well, thank you! I can assist with various tasks like answering questions, providing information, performing calculations, and more. For example, I can help multiply 

In [73]:
query = HumanMessage('can you multiply 3 with 10')

In [74]:
messages = [query]

In [75]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [76]:
result = c_llm_with_tools.invoke(messages)

In [77]:
messages.append(result)

In [78]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': '0', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 196, 'total_tokens': 224}, 'model_name': 'HuggingFaceH4/zephyr-7b-beta', 'system_fingerprint': '3.2.1-sha-4d28897', 'finish_reason': 'stop', 'logprobs': None}, id='run--09ba4396-b89c-4048-a1a3-7f885422fe16-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 196, 'output_tokens': 28, 'total_tokens': 224})]

In [79]:
result.tool_calls[0]['args']

{'a': 3, 'b': 10}

In [80]:
tool_result = multiply.invoke(result.tool_calls[0])

In [81]:
messages.append(tool_result)

In [82]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': '0', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 196, 'total_tokens': 224}, 'model_name': 'HuggingFaceH4/zephyr-7b-beta', 'system_fingerprint': '3.2.1-sha-4d28897', 'finish_reason': 'stop', 'logprobs': None}, id='run--09ba4396-b89c-4048-a1a3-7f885422fe16-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 196, 'output_tokens': 28, 'total_tokens': 224}),
 ToolMessage(content='30', name='multiply', tool_call_id='0')]

In [83]:
c_llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply', 'description': None}, 'id': '0', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 96, 'total_tokens': 126}, 'model_name': 'HuggingFaceH4/zephyr-7b-beta', 'system_fingerprint': '3.2.1-sha-4d28897', 'finish_reason': 'stop', 'logprobs': None}, id='run--d9678e82-f74a-4db7-a708-f0bc468d1a42-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 30, 'total_tokens': 126})